# GMC Regulates Edinburgh Medical School

## Note this might actually be the Medical Schools Board

![](https://www.medschools.ac.uk/media/2079/msc-logo.jpg)


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://general-medical-council-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and fill in additional logic as needed.

In [4]:
listeners = []

In [5]:
## YOUR LISTENERS HERE
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")
    if state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [6]:
def issuer_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    
    
    if state == "offer_sent":
        proposal = payload["credential_proposal_dict"]
        attributes = proposal['credential_proposal']['attributes']

        print(f"Offering credential with attributes  : {attributes}")
        ## YOUR LOGIC HERE
    elif state == "request_received":
        print("Request for credential received")
        ## YOUR LOGIC HERE
    elif state == "credential_sent":
        print("Credential Sent")
        ## YOUR LOGIC HERE
    
issuer_listener = {
    "topic": "issue_credential",
    "handler": issuer_handler
}

listeners.append(issuer_listener)

In [7]:
agent_controller.register_listeners(listeners)

## Store Issuing Schema and Cred Def Identifiers

If you intend for this agent to issue credentials you should first initialise your agent as an issuer and author the relevant identifiers to the public ledger. The issuer_initialisation recipe notebook can be duplicated and used as a starting point.

Once schema and cred def identifiers are created copy across and store in variables as illustrated in the cell below. Be sure to use unique names for each variable.

In [8]:
tms_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:Trusted School:0.0.6'
tms_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:14:institution'



%store tms_schema_id
%store tms_cred_def_id

Stored 'tms_schema_id' (str)
Stored 'tms_cred_def_id' (str)


## Establish Connection

Before you can issue a credential you must first establish a connection across which the credential will be issued to a holder. (see recipes/connection)

In [11]:
# Alias for invited connection
alias = "Medical School"
auto_accept = "true"
# Use public DID?
public = "true"
# Should this invitation be usable by multiple invitees?
multi_use = "false"

invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)
# Is equivalent to above. Arguments are optionally
# invitation_response = await agent_controller.connections.create_invitation()



# You will use this identifier to issue a credential across this connection
connection_id = invitation_response["connection_id"]

ERROR:__name__:Error during POST /connections/create-invitation: 403, message='Configuration does not include public invitations', url=URL('http://general-medical-council-agent:3021/connections/create-invitation?alias=Medical+School&auto_accept=true&public=true&multi_use=false')


ClientResponseError: 403, message='Configuration does not include public invitations', url=URL('http://general-medical-council-agent:3021/connections/create-invitation?alias=Medical+School&auto_accept=true&public=true&multi_use=false')

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  b89e0dcc-3669-45ef-8d10-65ea709445af
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  b89e0dcc-3669-45ef-8d10-65ea709445af
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  b89e0dcc-3669-45ef-8d10-65ea709445af
State :  active
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
Connection ID: b89e0dcc-3669-45ef-8d10-65ea709445af is now active.


## Share Invitation Object with External Agent

In [10]:
invitation = invitation_response["invitation"]
## Copy this output
print(invitation)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'efaf5f5d-af14-4a05-8368-9d99c354a846', 'recipientKeys': ['53cgX1XzkHFWLMM2PtG6zdfduQwYi635szbSATKJtSht'], 'label': 'General Medical Council', 'serviceEndpoint': 'http://f7f8334d4fa8.ngrok.io'}


## Display Invite as QR Code

This is useful if you wish to issue a credential to a mobile wallet.

In [ ]:
import qrcode
# Link for connection invitation
invitation_url = invitation_response["invitation_url"]
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=5,
        border=5)
qr.add_data(invitation_url)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img

## Populate Credential Attributes

Before you can issue a credential, you must define the values that will be issued in this credential. The attribute names **MUST** match those in the schem identified by the <schema_id> value.

Make sure to change all code enclosed with <>.


In [12]:
from datetime import date, timedelta

name="Edinburgh Medical School" # input("Please enter the value for <attr_1>: ")
date_issued=date.today().isoformat()
revalidation_due = (date.today() + timedelta(days=1825)).isoformat()
credential_attributes = [
    {"name": "Name", "value": name},
    {"name": "Date Issued", "value": date_issued},
    {"name": "Re-Validation Due", "value": revalidation_due}
]
print(credential_attributes)

[{'name': 'Name', 'value': 'Edinburgh Medical School'}, {'name': 'Date Issued', 'value': '2021-06-18'}, {'name': 'Re-Validation Due', 'value': '2026-06-17'}]


## Send Credential

This is the easiest way to issue a credential because it automates the rest of the protocol steps. 

Note: The `connection_id` must be in the active state before a credential can be sent.

In [16]:
# Do you want the ACA-Py instance to trace it's processes (for testing/timing analysis)
trace = True
comment = ""
# Remove credential record after issued?
auto_remove = False

# Change <schema_id> and <cred_def_id> to correct pair. Cred_def_id must identify a definition to which your agent has corresponding private issuing key.
send_cred_response = await agent_controller.issuer.send_credential(connection_id, tms_schema_id, tms_cred_def_id, credential_attributes, comment, auto_remove, trace)

# Note last three args are optional.
# await agent_controller.issuer.send_credential(connection_id, <schema_id>, <cred_def_id, credential_attributes)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : b89e0dcc-3669-45ef-8d10-65ea709445af
Credential exchange ID : f764c8bf-de80-452b-87a7-9050a098b7c9
Agent Protocol Role :  issuer
Protocol State :  offer_sent

---------------------------------------------------

Offering credential with attributes  : [{'name': 'Name', 'value': 'Edinburgh Medical School'}, {'name': 'Date Issued', 'value': '2021-06-18'}, {'name': 'Re-Validation Due', 'value': '2026-06-17'}]

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : b89e0dcc-3669-45ef-8d10-65ea709445af
Credential exchange ID : f764c8bf-de80-452b-87a7-9050a098b7c9
Agent Protocol Role :  issuer
Protocol State :  request_received

---------------------------------------------------

Request for credential received

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : b89e0dcc-3669-45ef-8d10-65ea709445a

## Optional: Revoke Credential

Only possible if the credential issued used a revocable credential definition. (see support_revocation flag in write_cred_def() function).

There are two approachs to revoke an revocable credential

### Option 1: Use credential Exchange ID

Note: In a SSI application as an issuer you are might want to keep track of the identifiers for credential echange records. These can be used to retrieve records stored by the agent and also be used to identify a credential to revoke
Note though that applications might not wish to retain these records as they contain sensitive PII

In [ ]:
record_id = send_cred_response["credential_exchange_id"]
try:
    record = await agent_controller.issuer.get_record_by_id(record_id)
except:
    print("Record not found. Did your agent automatically remove it?")

if record:
    # For revoke_credential() you only need to provide (cred_ex_id) OR (rev_reg_id AND cred_rev_id).
    response = await agent_controller.revocations.revoke_credential(record_id, publish=True)
    response

### Option 2: Remove Credential using the Registry and Credential Revocation ID

Does not currently work because of bug in code

In [ ]:
registry = await agent_controller.revocations.get_active_revocation_registry_by_cred_def(cred_def_id)
print(registry)

In [ ]:
revoc_reg_id = registry["result"]["revoc_reg_id"]

In [ ]:
# Public revocation registry update to ledger
publish = True
# Believe this is index in rev registry starting from 1
cred_rev_id = 1
response = await agent_controller.revocations.revoke_credential(cred_rev_id = cred_rev_id, rev_reg_id = revoc_reg_id, publish=True)

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [17]:
await agent_controller.terminate()